**Introduction to TensorFlow**
TensorFlow is an open source library for numerical computation using data flow graphs. Nodes in the graph represent mathematical operations (ops), while the graph edges represent the multidimensional data arrays (tensors) communicated between them. Tensors are the primary data structure that TensorFlow uses to operate on the computational graphs. Tensors are multi-dimensional arrays (similar to NumPy arrays) with a uniform type (called a dtype `tf.dtypes.DType`). Note that tensors are immutable like Python numbers and cannot be updated, we can only create a new one.

For a comprehensive guide see: https://www.tensorflow.org/guide


**Main ways of creating Tensors**

1. Fixed Tensors
    * `zero_tsr = tf.zeros([row_dim, col_dim])`
    * `ones_tsr = tf.ones([row_dim, col_dim])`
    * `filled_tsr = tf.fill([row_dim, col_dim], 42)`
    * `constant_tsr = tf.constant([1,2,3])`
2. Tensors of similar shape
    * `zeros_similar = tf.zeros_like(constant_tsr)`
    * `ones_similar = tf.ones_like(constant_tsr)`
3. Sequence Tensors
    * `linear_tsr = tf.linspace(start=0, stop=1, snum=100)`
    * `integer_seq_tsr = tf.range(start=6, limit=15, delta=3)`
4. Random Tensors
    * `randunif_tsr = tf.random.uniform([row_dim, col_dim], minval=0, maxval=1)`
    * `randnorm_tsr = tf.random.normal([row_dim, col_dim],mean=0.0, stddev=1.0)`


In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(tf.executing_eagerly()) ## this is a major shift from TF 1


ct1 = tf.constant(4)
print(ct1)

ct2 = tf.constant([[1, 2],[3, 4],[5, 6]], dtype=tf.float16)
print(ct2)

zt = tf.zeros([2, 3, 2], dtype = tf.int32)
ot = tf.ones([4, 2])

print(tf.reduce_sum(ot))
print(tf.reduce_sum(ot).numpy())
print(zt.numpy()) 


2.5.0-rc3
True
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16)
tf.Tensor(8.0, shape=(), dtype=float32)
8.0
[[[0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]]]


2022-11-08 12:40:02.631042: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Tensors may have more axes**

<img style="float: center;" src="./images/Tensors.png" width="600">

In [31]:
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_3_tensor.numpy())
print(rank_3_tensor.shape)

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]]
(3, 2, 5)


**TensorFlow Variables**

Variables are in-memory buffers containing tensors. All tensors we’ve used previously have been constant
tensors, not variables. Once tensors are created, then we may also create the corresponding variables by wrapping the tensor in the `Variable()` function: `my_var = tf.Variable(tf.zeros([row_dim, col_dim]))`. 

The main way to create a variable is by using the `Variable()` function, which takes a tensor as an input and outputs a variable. Variables are created and tracked via the `tf.Variable` class. A `tf.Variable` represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor.

In [32]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())


print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, ([1,4])))

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]
A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


As noted above, variables are backed by tensors. You can reassign the tensor using `tf.Variable.assign`. Calling assign does not allocate a new tensor; instead, the existing tensor's memory is reused.

In [33]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) ## This will not change the cast from tf.variable. a = tf.consts([1,2]) will recast a to tf.tensor

# Not allowed as it resizes the variable: 
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
# Note: a = a + tf.const([2.0, 3.0]) will change the cast of a from tf.Variable to tf.tensor
print(a.assign_add([2,3]).numpy())  # this will add and update a
print(a.assign_sub([7,9]).numpy())  # this will subtract and update a

ValueError: Shapes (2,) and (3,) are incompatible
[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


In [34]:
a = tf.Variable(0, dtype = tf.float32)

for _ in range(10):
    a.assign_add(1) ## a = a + 1


In [36]:
print(a.numpy())

10.0
